In [2]:
# import scapy
# !ls
import sys
sys.executable
import os 
import numpy
import time
import threading

# print('hello')

In [16]:
prev_seq_num = -1
wifi_path = '/home/hu/Desktop/memory_file_sys/wifi.pcap'


print('---begin reading pcap---')


filesize = os.path.getsize(wifi_path)
print(filesize)
with open(wifi_path, 'rb') as in_wifi:

    in_wifi.seek(-2, os.SEEK_END)
    while in_wifi.read(1) != b"\n":
        in_wifi.seek(-2, os.SEEK_CUR)
    l = in_wifi.readline()
    
    print(l[-22:])
    l = l[-22:].decode().strip()
    print('read line : ', l)
    in_wifi.close()
    
    packet_data = l
    in_wifi.close()
    packet_data = packet_data.split(' ')
    print(packet_data, type(packet_data))
    source_vel = packet_data[0]
    cur_seq_num = int(packet_data[1])
    rel_dist = float(packet_data[2])
    vel_leader = float(packet_data[3])
    print(f'read from pcap {source_vel} {cur_seq_num}, {rel_dist}, {vel_leader}')
    print(f'cur_seq_num = {cur_seq_num} while prev_seq_num = {prev_seq_num}')
    if (cur_seq_num != prev_seq_num):
        packet_flag = False
    prev_seq_num = cur_seq_num

    

---begin reading pcap---
24


OSError: [Errno 22] Invalid argument

In [ ]:
# infile = open('../wifi.pcap', 'rb')
# cnt = 0
# a = infile.read(40)

def read_msg(infile, length):
    print('---message begin---')
    for i in range(length):
        a = infile.read(1).decode('ascii')
        print(a, end = ' ')
        if i % 20 == 19:
            print()
    print()

with open('../wifi.pcap', 'rb') as infile:
    
    flag = True
    while flag:
        former = '00'
        a = '00'
        length, cnt = 0, 0
        print('---packet begin---')
        for i in range(500):
            former = a
            a = infile.read(1).hex()
            length += 1
            if a is None:
                flag = False
                break

            if former == 'ff' and a == 'ff':
                # print(f'cnt = {cnt}')
                cnt += 1
            else:
                cnt = 0
            print(a, end=' ')
            # print(type(a))
            if i % 10 == 9:
                print()
            if cnt == 5:
                a = infile.read(2).hex()
                print(a[:2] + ' ' + a[2:])
                if a == 'ffff':
                    flag = False
                    break
                read_msg(infile, 400)
                # print(msg)
                print(f'length = {length}')
                cnt = 0
                length = 0
                break

In [5]:
def read_latest_packet(file_path = '../wifi.pcap', packet_len = 457):
    while(1):
        with open(file_path, 'rb') as infile:
            
            infile.seek(-packet_len, 2)
            print(infile.read(packet_len))
            infile.close()
        time.sleep(1)

# read_latest_packet()


In [7]:
import os
import time
infile = open('/home/hfc/Desktop/wifi.pcap', 'rb')
while 1:
    infile.seek(-457, 2)
    a = infile.readline().hex()
    print(a)
    time.sleep(1)

0d681164b4430d00b9010000b9010000000011006e080000000cb200000091000108000000424242424242232323232323ffffffffffff000861616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161616161
0d681164b4430d00b9010000b9010000000011006e080000000cb20000009100010800000042424242424

KeyboardInterrupt: 

In [ ]:
try:

    threading.Thread(target=read_latest_packet, args=()).start()
except Exception as e:
    print("---异常---：", e)

In [ ]:
import struct
 
class FileConvert(object):
    ''' test python file'''
    def __init__(self):
        self.aa = 0
        self.bb = 0
        
    def convertPcapToHex(self,debugFlg=0):
        infile = open("test.pcap",'rb')
        outfile = open("testRslt.txt",'w')
        a = int(0)
 
        while True:
            a = int(0)
            a = infile.read(4)
            #print(type(a))
            #print(len(a))
            dataSaveNum = len(a)
            if not a:
                break
            if(len(a) == 4):
                pass
            elif(len(a) != 0):
                loopTime = 4 - len(a)
                while (loopTime > 0):
                    loopTime = loopTime - 1
                    a = a + b'\x00'
            z = struct.unpack('i',a)
            unNeg = z[0] & 0xFFFFFFFF #负数的验证
            if(dataSaveNum == 4):
                outfile.write("%08x"%unNeg)
                dataSaveNum = 0
            
            while(dataSaveNum > 0):
                saveData = unNeg&0xFF
                #保留单个字节
                #outfile.write("%08x"%unNeg)
                unNeg = unNeg >> 8
                dataSaveNum = dataSaveNum - 1
                outfile.write("%02x"%saveData)
                
            if(debugFlg == 1):
                outfile.write("\n")
                
        infile.close()
        outfile.close()
      
    def convertHexToPcap(self):
        infile = open("testRslt.txt",'r')
        outfile = open("another.pcap",'wb')
        #for a in infile.read():
        while True:
            a = infile.read(8)
            #print(type(a))
            #print(b);
            if not a:
                break
            if(len(a)!= 0):
                #16进制转换10进制数
                b=int(a,16)
                covertValue = struct.pack("L",b)
                outfile.write(covertValue)
            #outfile.write("%s"%a)
            #outfile.write("\n")
        infile.close()
        outfile.close()
 
if __name__ == '__main__':
    fileCvrt = FileConvert()
    fileCvrt.convertPcapToHex()
    fileCvrt.convertHexToPcap()